## 0. Import des librairies

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
import keras
from keras import metrics
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation
import os
import pickle
import import_auto


Using TensorFlow backend.


In [2]:
import_auto.import_fichier_train()

In [3]:
import_auto.import_fichier_test()

## 1. Import des Datasets

In [ ]:
# ImageDataGenerator génère des lots de données d'image vectorielles, convertissant les coefficients RVB compris entre 0 et 255 en valeurs cibles comprises entre 0 et 1 par mise à l'échelle avec un facteur de 1/255 à l' aide de la remise à l' échelle

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,           # shear_range est utilisé pour appliquer de manière aléatoire des transformations de cisaillement
                                   zoom_range = 0.2,            # zoom_range est utilisé pour zoomer aléatoirement à l'intérieur des images
                                   horizontal_flip = True)      # horizontal_flip est utilisé pour retourner au hasard la moitié des images horizontalement

test_datagen = ImageDataGenerator(rescale = 1./255)


# Ensuite, nous importons les images une par une à partir des répertoires en utilisant .flow_from_directory et y appliquons ImageDataGenerator.
train_generator = train_datagen.flow_from_directory(
    directory = "Training",                                     # Choix de mon repertoire
    target_size = (32,32),                                      # Nous convertissons ensuite les images de leur taille d'origine à notre target_size
    batch_size = 32,                                            # Nombre batch_size qui fait référence au nombre d'exemples d'entraînement utilisés dans une                                                                        itération
    class_mode = "categorical"                                  # Nous définissons le class_mode sur catégorique indiquant que nous avons plusieurs classes (a à z)                                                                 à prédire.

)

test_generator = test_datagen.flow_from_directory(
    directory = "Testing",
    target_size = (32,32),
    batch_size = 32,
    class_mode = "categorical"

)

## 2. Definition du modèle d'architecture

In [ ]:
# Création d'un modèle séquentiel qui permet de définir l'architecture CNN couche par couche à l'aide de la fonction .add .Nous ajoutons d'abord une couche de convolution avec 32 filtres de taille 3X3 sur les images d'entrée et la passons à travers la fonction d'activation 'relu'.Nous effectuons ensuite des opérations MaxPooling en utilisant un pool de taille 2X2 
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape = (32,32,3), activation = "relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))

# Ces couches sont ensuite répétées à nouveau pour améliorer les performances du modèle.

model.add(Conv2D(32, (3, 3), activation = "relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))

# Enfin, nous aplatissons notre matrice résultante et la passons à travers une couche dense composée de 128 nœuds. Celui-ci est ensuite connecté à la couche de sortie constituée de 26 nœuds, chaque nœud représentant un alphabet

model.add(Flatten())
model.add(Dense(units = 128, activation = "relu"))
model.add(Dense(units = 27, activation = "softmax"))        # Nous utilisons l' activation softmax qui convertit les scores en une distribution de probabilité                                                                   normalisée, et le nœud avec la probabilité la plus élevée est sélectionné comme sortie

# Une fois notre architecture CNN définie, nous compilons le modèle à l'aide de l'optimiseur Adam 
model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])

model.summary()

## 3. Entrainement du modèle

In [ ]:
### Je décide de créer 30 répétitions, et j'ai X2 les steps_per_epoch pour augmenter notre précision ###
entrainement = model.fit_generator(train_generator,
                         steps_per_epoch = 32,
                         epochs = 30,
                         validation_data = test_generator,
                         validation_steps = 32)

score = model.evaluate(train_generator, verbose=0)
print('Test de perte:', score[0])
print('Test de précision:', score[1])


## 4. Enregistrement & chargement du modèle

In [ ]:
pickle.dump(model, open("mon_model.sav", "wb"))
model = pickle.load(open("mon_model.sav","rb"))

## 5. Création d'une fonction qui renvois l'alphabet associé au résultat

In [ ]:
def get_result(result):
    if result[0][0] == 1:
        return("a")
    elif result[0][1] == 1:
        return ("b")
    elif result[0][2] == 1:
        return ("c")
    elif result[0][3] == 1:
        return ("d")
    elif result[0][4] == 1:
        return ("e")
    elif result[0][5] == 1:
        return ("f")
    elif result[0][6] == 1:
        return ("g")
    elif result[0][7] == 1:
        return ("h")
    elif result[0][8] == 1:
        return ("i")
    elif result[0][9] == 1:
        return ("j")
    elif result[0][10] == 1:
        return ("k")
    elif result[0][11] == 1:
        return ("l")
    elif result[0][12] == 1:
        return ("m")
    elif result[0][13] == 1:
        return ("n")
    elif result[0][14] == 1:
        return ("o")
    elif result[0][15] == 1:
        return ("p")
    elif result[0][16] == 1:
        return ("q")
    elif result[0][17] == 1:
        return ("r")
    elif result[0][18] == 1:
        return ("s")
    elif result[0][19] == 1:
        return ("t")
    elif result[0][20] == 1:
        return ("u")
    elif result[0][21] == 1:
        return ("v")
    elif result[0][22] == 1:
        return ("w")
    elif result[0][23] == 1:
        return ("x")
    elif result[0][24] == 1:
        return ("y")
    elif result[0][25] == 1:
        return ("z")
    elif result[0][26] == 1:                # Création du symbole <3
        return ("<3")

## 6. Choix de l'image à envoyer

In [ ]:
choix = r"Testing\z_coeur\COEUR-thomas.png"
test_image = image.load_img(choix, target_size = (32,32))
plt.imshow(test_image)
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)


## 7. Test du modèle

In [ ]:
resultat = model.predict(test_image)
resultat = get_result(resultat)
print(f"La lettre correspondante est: {resultat}")

In [ ]:
score = model.evaluate(test_generator, verbose=1)
print('Test de perte:', score[0])
print('Test de précision:', score[1]) 

In [ ]:
# predict_proba().